# SerpAPI

[SerpAPI](https://serpapi.com/) 允许您将搜索引擎结果集成到您的 LLM 应用中

本指南简要介绍了如何开始使用 SerpAPI [工具](/docs/integrations/tools/)。如需了解 `SerpAPI` 所有功能和配置的详细文档，请前往 [API 参考文档](https://api.js.langchain.com/classes/_langchain_community.tools_serpapi.SerpAPI.html)。

## 概览

### 集成详情

| 类 | 包 | [PY 支持](https://python.langchain.com/docs/integrations/tools/serpapi/) | 包最新版本 |
| :--- | :--- | :---: | :---: |
| [SerpAPI](https://api.js.langchain.com/classes/_langchain_community.tools_serpapi.SerpAPI.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 安装设置

该集成位于 `@langchain/community` 包中，您可以按照以下方式安装：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>
```

### 凭据

在此处 [设置 API 密钥](https://serpapi.com/)，并将其设置为名为 `SERPAPI_API_KEY` 的环境变量。

```typescript
process.env.SERPAPI_API_KEY = "YOUR_API_KEY"
```

此外，设置 [LangSmith](https://smith.langchain.com/) 以获得最佳观测能力也很有帮助（但不是必须的）：

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

你可以通过以下方式导入并实例化 `SerpAPI` 工具：

In [1]:
import { SerpAPI } from "@langchain/community/tools/serpapi";

const tool = new SerpAPI();

## 调用

### [使用参数直接调用](/docs/concepts/#invoke-with-just-the-arguments)

你可以像这样直接调用工具：

In [2]:
await tool.invoke({
  input: "what is the current weather in SF?"
});

{"type":"weather_result","temperature":"63","unit":"Fahrenheit","precipitation":"3%","humidity":"91%","wind":"5 mph","location":"San Francisco, CA","date":"Sunday 9:00 AM","weather":"Mostly cloudy"}


### [使用 ToolCall 调用](/docs/concepts/#invoke-with-toolcall)

我们还可以通过模型生成的 `ToolCall` 来调用工具，在这种情况下将返回一个 `ToolMessage`：

In [3]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    input: "what is the current weather in SF?"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}

await tool.invoke(modelGeneratedToolCall)

ToolMessage {
  "content": "{\"type\":\"weather_result\",\"temperature\":\"63\",\"unit\":\"Fahrenheit\",\"precipitation\":\"3%\",\"humidity\":\"91%\",\"wind\":\"5 mph\",\"location\":\"San Francisco, CA\",\"date\":\"Sunday 9:00 AM\",\"weather\":\"Mostly cloudy\"}",
  "name": "search",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_call_id": "1"
}


## 链式调用

我们可以通过首先将其绑定到一个[工具调用模型](/docs/how_to/tool_calling/)，然后调用它，以链式方式使用我们的工具：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [4]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
})

In [6]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("what is the current weather in sf?");

In [7]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

AIMessage {
  "tool_calls": [],
  "content": "The current weather in San Francisco is mostly cloudy, with a temperature of 64°F. The humidity is at 90%, there is a 3% chance of precipitation, and the wind is blowing at 5 mph."
}


## 代理

关于如何在代理中使用LangChain工具的指南，请参阅 [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) 文档。

## API 参考文档

有关所有 `SerpAPI` 功能和配置的详细文档，请访问 API 参考页面: https://api.js.langchain.com/classes/_langchain_community.tools_serpapi.SerpAPI.html